# Mise en oeuvre du modèle Mistral Large (123B paramètres ?)

Plus de détails sur le modèle : https://mistral.ai/fr/news/mistral-large/

La partie RAG suit le traitement proposé ici : https://docs.mistral.ai/guides/rag/

Il est possible d'affiner (fine tune) ce type de modèle, y compris via l'API si on n'a pas assez de ressources de calcul en local, mais ce ne sera pas vu dans ce cours : https://docs.mistral.ai/guides/finetuning/

Ce qui suit est une démonstration de l'utilisation de l'API de Mistral, comme alternative aux modèles de référence comme GPT. A noter qu'il est possible, en plus de faire de la génération à partir de prompt et des données de préentrainement du modèle, de suivre le principe RAG (Retrieval Augmented Generation) afin de coupler ça avec une base de données qui a été indexée.

In [38]:
# librairies utilisées
# !pip install mistralai faiss-gpu # if you use GPU
# !pip install mistralai faiss-cpu # if you use CPU

import os
import numpy as np
import time
import pickle

Tout d'abord, il s'agit de charger la clef d'accès à l'API de Mistral (gratuit pour une utilisation individuelle mais potentiellement limitée).

In [21]:
import os

api_key = os.environ["MISTRAL_KEY_API"]

Après avoir [installé la librairie *mistralai*](https://docs.mistral.ai/getting-started/clients/), vous pouvez la charger en mémoire et choisir votre modèle.

In [22]:
from mistralai import Mistral

# choix du modèle (ici la version large)
model = "mistral-large-latest"

client = Mistral(api_key=api_key)

Pour commencer, quelques tests simples d'autocomplétion, en anglais...

In [23]:
chat_response = client.chat.complete(
    model= model,
    messages = [
        {
            "role": "user", # "rôle" que va jouer le robot (liste à choix fermée)
            "content": "What is the best French cheese?", # la question
        },
    ]
)
print(chat_response.choices[0].message.content)

Choosing the "best" French cheese can be subjective, as it greatly depends on personal taste. France is renowned for its wide variety of cheeses, with over 400 different types. Here are a few highly regarded French cheeses across various categories:

1. **Soft Cheeses**:
   - **Brie de Meaux**: Known for its creamy texture and rich, slightly earthy flavor.
   - **Camembert de Normandie**: Soft and creamy with a distinctive, pungent aroma.

2. **Semi-Soft Cheeses**:
   - **Munster**: A strong-smelling, wash-rind cheese with a powerful flavor, originating from the Alsace region.

3. **Hard Cheeses**:
   - **Comté**: A firm, nutty cheese made from unpasteurized cow's milk, often compared to Gruyère.
   - **Beaufort**: Similar to Comté, with a firm texture and complex, nutty flavor.

4. **Blue Cheeses**:
   - **Roquefort**: A tangy, salty blue cheese made from sheep's milk, known for its distinctive veins of blue mold.

5. **Goat Cheeses**:
   - **Chèvre**: French goat cheese comes in many

...et en français :

In [24]:
chat_response_fr = client.chat.complete(
    model= model,
    messages = [
        {
            "role": "user",
            "content": "La réponse s'adresse à un locuteur français. Quel est le meilleur fromage ?",
        },
    ]
)
print(chat_response_fr.choices[0].message.content)

La question du "meilleur fromage" est très subjective et dépend des goûts personnels de chacun. La France, en particulier, est célèbre pour sa grande variété de fromages, chacun ayant ses propres caractéristiques et saveurs uniques. Voici quelques fromages français très appréciés :

1. **Roquefort** : Un fromage persillé au lait de brebis, connu pour son goût fort et sa texture crémeuse.
2. **Camembert** : Un fromage à pâte molle et à croûte fleurie, souvent apprécié pour sa douceur et son goût prononcé.
3. **Brie de Meaux** : Un autre fromage à pâte molle et à croûte fleurie, célèbre pour sa saveur riche et crémeuse.
4. **Comté** : Un fromage à pâte pressée cuite, au goût fruité et noiseté, qui peut varier en fonction de son âge.
5. **Reblochon** : Un fromage à pâte pressée non cuite, souvent utilisé dans la tartiflette, avec un goût doux et crémeux.

Il existe des centaines de fromages différents, et chacun a ses préférences. Le meilleur fromage est donc celui que vous appréciez le p

On peut aussi demander à un modèle proche de fournir les plongements (*embeddings*) de textes :

In [25]:
model_emb = "mistral-embed"

client = Mistral(api_key=api_key)

embeddings_response = client.embeddings.create(
    model=model_emb,
    inputs=["Un fromage à pâte pressée cuite, originaire du Jura et de la Franche-Comté.",
            "Il existe de nombreuses variétés de fromages de chèvre, comme le Crottin de Chavignol ou le Sainte-Maure de Tourain.",
           "Le pithiviers est un gâteau français à base de pâte feuilletée originaire de la commune de Pithiviers située dans le département du Loiret et la région Centre-Val de Loire.",
           "Le château de Versailles est un château et un monument historique français situé à Versailles, dans les Yvelines."]
)

In [26]:
vectors = [np.array(e.embedding) for e in embeddings_response.data]
print(len(vectors))

4


In [27]:
# similarité cosinus entre deux vecteurs sous numpy
def cosine(x, y):
    return float(cosine_similarity(x.reshape(1, -1), y.reshape(1, -1)).squeeze())

In [28]:
print(f"Similarité cosinus : {cosine(vectors[0], vectors[1]):.3f} ")

Similarité cosinus : 0.830 


# Combiner Mistral avec du RAG

Le RAG (*Retrieval Augmented Generation*) permet de combiner la recherche d'information (pour trouver les meilleurs passages) et la génération d'une réponse.

La solution présentée ici ne recourt pas à des librairies comme Langchain ou LlamaIndex, que nous ne verrons pas en cours, mais beaucoup de ressources en ligne en parlent très bien (comme : https://docs.mistral.ai/guides/rag/).

Chargement des données textuelles à partir d'un fichier (cf. début du cours) :

In [29]:
with open("alice.txt") as f:    
    lines = [line.strip() for line in f.readlines()]

print(f"{len(lines)} lignes dans le fichier texte")

5777 lignes dans le fichier texte


On combine les lignes pour produire un corpus de paragraphes (il faut éviter les documents vides).

In [30]:
docs = []
s = ""
for l in lines:
    if l:
        s += " " + l
    elif s:
        docs.append(s)
        s = ""

print(f"{len(docs)} paragraphes")

2033 paragraphes


On choisit le modèle d'embedding et on initialise le modèle (comme tout à l'heure).

In [31]:
model_emb = "mistral-embed"

client = Mistral(api_key=api_key)

Si la base de données vectorielle n'a pas été créée, il faut le faire (cela peut prendre du temps). Attribuer alors valeur *True* à la variable *build_embeddings* pour lancer le travail avec l'API.

In [34]:
build_embeddings = False  # mettre True pour recalculer tous les plongements

if build_embeddings:
    text_embeddings = []
    docs_2_process = docs.copy()
    step = 512  # nombre de documents traités à la fois (à priori, max 512 pour Mistral large)

    def process_docs(docs_batch):
        embeddings_response = client.embeddings.create(
            model=model_emb,
            inputs=docs_batch
        )
        return [np.array(e.embedding) for e in embeddings_response.data]

    while docs_2_process:
        liste_docs = docs_2_process[:step]
        text_embeddings.extend(process_docs(liste_docs))
        docs_2_process = docs_2_process[step:]
        print(len(liste_docs))
        time.sleep(5)  # petit temps de latence pour ne pas surcharger l'API

    vectors_list = np.array(text_embeddings)

    with open("save_emb_alice.pkl", "wb") as f:
        pickle.dump(vectors_list, f)

In [33]:
if not build_embeddings:
    with open("save_emb_alice.pkl", "rb") as f:
        vectors_list = pickle.load(f)

In [36]:
print(vectors_list.shape)

(2033, 1024)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from faiss import IndexFlatIP
import faiss.contrib.torch_utils

A partir de là, on peut créer l'index de la base de données vectorielle grâce à la libaririe *faiss* :

In [47]:
vectors_list_f32 = np.array(vectors_list, dtype=np.float32)

dim = vectors_list_f32.shape[1]
index = faiss.IndexFlatL2(dim)
print(dim, index.is_trained)
index.add(vectors_list_f32)

1024 True


Note : pour accélérer les choses, le mieux serait de sauvegarder cet index sur le disque et pas de le recalculer à chaque fois.

# Poser des questions sur les données textuelles

A présent, on peut utiliser l'index pour poser n'importe quelle question.

In [ ]:
def poser_une_question(question):
    def create_embedding(input_text):
        response = client.embeddings.create(
            model=model_emb,
            inputs=input_text
        )
        return np.array([response.data[0].embedding])

    def retrieve_documents(embedding, k=2):
        _, indices = index.search(embedding, k)
        print(data)
        print([docs[i] for i in indices.tolist()[0]])
        return [docs[i] for i in indices.tolist()[0]]

    question_embedding = create_embedding(question)
    retrieved_docs = retrieve_documents(question_embedding)

    messages = [
        {
            "role": "user",
            "content": f"""Context information is below.
            ---------------------
                {retrieved_docs}
            ---------------------
            Given the context information and not prior knowledge, answer the query.
                Query: {question}
            Answer:"""
        }
    ]

    time.sleep(5)

    chat_response = client.chat.complete(
        model="mistral-large-latest",
        messages=messages
    )

    return chat_response.choices[0].message.content

In [52]:
question1 = "What unexpected animal the duchess comes out carrying in baby's clothes?"
print(poser_une_question(question1))

[[0.33786166 0.3802628 ]]
[" [_She starts to open the door just as the DUCHESS comes out carrying a pig in baby's clothes. She sneezes--FROG sneezes and ALICE sneezes._]", ' The Duchess!']
A pig


In [50]:
question2 = "What is the usual fate reserved for the queen's enemies?"
print(poser_une_question(question2))

Based on the context provided, the usual fate reserved for the queen's enemies is to be beheaded. This is indicated by the statement: "You'd better not talk. I heard the Queen say only yesterday you deserved to be beheaded."


Les réponses apportée semblent tout à fait pertinentes.